<a href="https://colab.research.google.com/github/nekoniii3/GPT_function_calling/blob/main/function_calling_%E5%A4%A9%E6%B0%97%E4%BA%88%E5%A0%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenAIパッケージのインストール
!pip install openai

In [ ]:
import openai

openai.api_key = "ここをOpenAIキーに入れ替えてください。" #@param {type:"string"}


In [ ]:
import json
import requests

# get_weather_info：地域コードから今日の天気を返す関数（APIの詳細は"https://weather.tsukumijima.net"）
def get_weather_info(city_code):

    url = "https://weather.tsukumijima.net/api/forecast/city/" + str(city_code)

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return json.dumps(data["forecasts"])
    else:
        return None

# Chat-GPTに伝えるための関数の仕様を記したのJSON
weather_function =  {
    "name": "get_weather_info",
      "description": "5桁の地区コード（一次細分区域）を元に天気予報を返す",
      "parameters": {
          "type": "object",
          "properties": {
              "city_code": {
                  "type": "string",
                  "description": "地区コード5桁（一次細分区域）"
              },
          },
          "required": ["city_code"]
      }
}


In [ ]:
# GPTへの問い合わせ ※たまにGPTが地域コード間違えるので注意
prompt = "東京の今日の天気は？"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": prompt}],
    functions=[weather_function],
    function_call="auto",
)

# GPTからのレスポンス取得
message = response["choices"][0]["message"]

# message内に"function_call"があるなら関数を実行し再びGPTへ問い合わせ
if message.get("function_call"):
    function_name = message["function_call"]["name"]
    arguments=json.loads(message["function_call"]["arguments"])
    function_response = get_weather_info(
        city_code=arguments.get("city_code")
    )
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": prompt},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # GPTからの回答を出力
    print(second_response.choices[0]["message"]["content"])